# **FYP-EEE-A3245-211**
# **Face Transformation using StyleGAN**

This is a Google Colab notebook for generating StyleGAN2 images.


---
# **0. Install/Import Prerequisites**

In [1]:
!pip install --upgrade --no-cache-dir gdown

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for gdown: filename=gdown-4.4.0-py3-none-any.whl size=14774 sha256=703ec5605922667d5b3c8adfb4b9e627341cc35ce97d8975fca0356631247e4b
  Stored in directory: /tmp/pip-ephem-wheel-cache-3m_ysxsy/wheels/fb/c3/0e/c4d8ff8bfcb0461afff199471449f642179b74968c15b7a69c
Successfully built gdown
  Attempting uninstall: gdown
    Found existing installation: gdown 4.2.2
    Uninstalling gdown-4.2.2:
      Successfully uninstalled gdown-4.2.2


In [2]:
!pip install ninja

     |████████████████████████████████| 108 kB 9.7 MB/s 


In [3]:
# clone StyleGAN2 repo
!git clone https://github.com/NVlabs/stylegan2-ada-pytorch

Cloning into 'stylegan2-ada-pytorch'...
remote: Enumerating objects: 128, done.
remote: Total 128 (delta 0), reused 0 (delta 0), pack-reused 128
Receiving objects: 100% (128/128), 1.12 MiB | 26.09 MiB/s, done.
Resolving deltas: 100% (57/57), done.


In [4]:
# download pretrained network (stylegan2-ffhq-config-f)
!gdown --id 1f9ZWi2iV26AMixY1dtO_tkc59oSCvDTS

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1f9ZWi2iV26AMixY1dtO_tkc59oSCvDTS
To: /content/stylegan2-ffhq-config-f.pkl
100% 382M/382M [00:03<00:00, 113MB/s]


# **1. StyleGAN Setup**

In [5]:
%cd /content/stylegan2-ada-pytorch

/content/stylegan2-ada-pytorch


In [6]:
import torch
import random
import warnings
import numpy as np

from tqdm import tqdm
from PIL import Image

import dnnlib
import legacy

In [7]:
# Generate latent vector
def generate_latent_vector(seed, shape):
  # Construct random number generator
  # Generate vector of size[1,512]
  rnd = np.random.RandomState(seed)
  z = rnd.randn(1, shape)
  return z

# Generate Image
def generate_image(z, label, truncation_psi=0.5, noise_mode="const"):
  z = torch.from_numpy(z).to(device)
  img_gpu = G(z, label, truncation_psi=truncation_psi, noise_mode=noise_mode)
  img_tensor = (img_gpu.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8)
  img = np.transpose(img_tensor.cpu().numpy()[0], (0,1,2))
  return img

In [8]:
# Get shape of latent vector and torch label
def setup_image_generation(G, device):
  z_shape = G.z_dim
  zero_label = torch.zeros([1, G.c_dim], device=device)
  # print('Setting up image generation...', end=' ')
  z = generate_latent_vector(0, z_shape)
  img = generate_image(z, zero_label)
  del z, img
  # print('Done')
  return z_shape, zero_label

# Check if cuda is available
def torch_is_cuda_available():
  if torch.cuda.is_available(): 
    print('CUDA is available. Using device \'0\': %s'%(torch.cuda.get_device_name(0)))
    return 'cuda'
  else:
    warnings.warn('Warning! Using CPU...')
    return 'cpu'

def load_network(path_to_network):
  print('Unpacking network file...', end=' ')
  with dnnlib.util.open_url(path_to_network) as f:
      G = legacy.load_network_pkl(f)['G_ema'].to(device)
  print('Done')
  return G

In [10]:
# Initialize network and setup for image generation
device = torch.device(torch_is_cuda_available())
path_to_network = '/content/stylegan2-ffhq-config-f.pkl'
G = load_network(path_to_network)
(z_shape, label) = setup_image_generation(G, device)
print('Ready')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: UserWarning: Warning! Using CPU...


Unpacking network file... No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
Done
Ready


---

# **2. Generate and Save Images**

In [11]:
# create folder to hold output
!mkdir /content/output

In [12]:
# path to output folder
path = '/content/output'

In [16]:
# Choose starting and ending seed
start = 0
end = 1000

In [19]:
for idx in tqdm(range(start, end, 1), desc='Generating Images...'):
    z = generate_latent_vector(idx, z_shape)  # generate latent vector
    img = generate_image(z, label)  # generate image (np)
    img = Image.fromarray(img)  # np to PIL conversion
    img.save(path + '/seed_%d.png'%(idx), 'PNG')  # save image as PNG
print('\nDone...')

Generating Images...: 100%|██████████| 5/5 [00:25<00:00,  5.10s/it]


Done...
